## 1. Expression of $ Z(\alpha, \beta)$

The partition function $ Z(\alpha, \beta) $ ensures that $ p(x) $ is a valid probability distribution:

$$
Z(\alpha, \beta) = \sum_{x \in \{0,1\}^n} \exp \left( \alpha \sum_i x_i + \beta \sum_{i \sim j} \mathbf{1}_{x_i = x_j} \right)
$$

where the sum runs over all possible configurations \( x \) of the system.

### Why is $ Z(\alpha, \beta)$ difficult to compute?
- The sum has **exponential complexity** in $ n $.
- Computing it exactly requires summing over $ 2^n $ configurations, which is infeasible for large $ n $.
- Instead, we use **sampling methods** (like Monte Carlo methods) to approximate it.






## Likelihood and Log-Likelihood 

Let $ x^1, x^2, \dots, x^n $ represent the **different observed configurations** of the Ising model, where each $ x^i $ is a vector of binary values representing the spin configurations.

### Likelihood Function

The **likelihood** is the **product of the probabilities** of these observed configurations given the model parameters $ \alpha $ and $ \beta $. For each configuration $ x^i $, the probability is given by the Ising model distribution:

$$
p(x^i \mid \alpha, \beta) = \frac{1}{Z(\alpha, \beta)} \exp \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right)
$$

The **likelihood** is the product of the probabilities for all observed configurations $ x^1, x^2, \dots, x^n $:

$$
L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \prod_{i=1}^{n} p(x^i \mid \alpha, \beta)
$$

Substituting the probability for each observation:

$$
L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \prod_{i=1}^{n} \frac{1}{Z(\alpha, \beta)} \exp \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right)
$$

### Log-Likelihood Function

To simplify the computation, we take the **logarithm** of the likelihood to obtain the **log-likelihood**:

$$
\log L(\alpha, \beta \mid x^1, x^2, \dots, x^n) = \sum_{i=1}^{n} \left( \alpha \sum_j x^i_j + \beta \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} \right) - n \log Z(\alpha, \beta)
$$

Where:
- $ x^i_j $ refers to the $ j $-th element (spin) of the $ i $-th configuration vector $ x^i $.
- The sum $ \sum_j x^i_j $ is the total sum of spins in the $ i $-th configuration.
- The sum $ \sum_{j \sim k} \mathbf{1}_{x^i_j = x^i_k} $ counts the number of neighboring pairs of spins that are equal in the $ i $-th configuration.
- $ Z(\alpha, \beta) $ is the partition function, which normalizes the distribution.

---

### Why is MLE difficult?
- Computing $ Z(\alpha, \beta)$ is **intractable**.
- Gradient-based optimization is challenging because computing gradients requires evaluating $ Z(\alpha, \beta)$.

👉 **Solution**: Instead of MLE, we use **Approximate Bayesian Computation (ABC)**.

------------------

In [1]:
from modules.ABC_reject import *
from modules.Gibbs_sampler import *

In [2]:
# Example run
n = 20  # Grid size
true_alpha, true_beta = 0.5, 0.8

observed_grid1 = run_gibbs(n, true_alpha, true_alpha, steps=1)
# observed_grid = ising_model(n, true_alpha, true_beta)
obs_stats1 = sufficient_statistics(observed_grid1)

In [3]:
# Example run
n = 20  # Grid size
true_alpha, true_beta = 0.5, 1.2

observed_grid2 = run_gibbs(n, true_alpha, true_alpha, steps=1)
# observed_grid = ising_model(n, true_alpha, true_beta)
obs_stats2 = sufficient_statistics(observed_grid2)

In [4]:
print(mse_distance(np.array(obs_stats1), np.array(obs_stats2)))
print(mae_distance(np.array(obs_stats1), np.array(obs_stats2)))

48.5
6.5


In [9]:
# Example run
n = 20  # Grid size


theta_samples = abc_reject(
    obs_stats1,
    prior_alpha=(0, 1),
    prior_beta=(0, 1),
    n=n,
    epsilon=5,
    num_samples=10000,
)


print("Estimated alpha and beta:", np.mean(theta_samples, axis=0))

min_idx = np.argmin(theta_samples[:, 2])

# Get the corresponding alpha and beta
best_alpha, best_beta, best_dis = (
    theta_samples[min_idx, 0],
    theta_samples[min_idx, 1],
    theta_samples[min_idx, 2],
)

print("nearest alpha and beta: ", (best_alpha, best_beta))
print("best distance: ", best_dis)
print(
    " MSE :",
    0.5
    * (
        abs(0.5 - np.mean(theta_samples, axis=0)[0])
        + abs(0.8 - np.mean(theta_samples, axis=0)[1])
    ),
)

0.953404544140044 0.5479844121502034 30
0.12828163824899685 0.37541664555143783 79
0.3655806832883711 0.4316661975866568 66
0.9211472460104664 0.41055727654695917 46
0.834216663094637 0.5321280223649234 31
0.11409315018179467 0.5718445440893386 49
0.20772863124784824 0.2904295040632081 46
0.7042206551489455 0.6674232023324012 9
0.3784433770078258 0.27328620259787184 34
0.8691930848793127 0.425661701049313 43
0.5176177368347613 0.9477554215279647 12
0.32587187103967796 0.8387111685941485 29
0.3292456009185465 0.2634028820757782 58
0.36987251247718955 0.30516450432167563 50
0.17237824282932135 0.9009947980130899 60
0.15043689019809214 0.49369387630630424 79
0.6268180272834265 0.4458309835467036 12
0.27298063398558037 0.18826730153691007 109
0.35030500754351224 0.5967563010559996 52
0.7127644029855436 0.20542453490360868 21
0.8644311773964433 0.9744236061159239 43
0.19364585450356586 0.008390432026450623 104
0.030934142888436322 0.6306069660295629 89
0.0838952364064417 0.6591487454975399 

# Approximate Bayesian Computation (ABC-Reject) for Ising Model

## 1. Understanding the Sufficient Statistics

In the Ising model, we define the **sufficient statistics**:

$ S(x) = \sum_i x_i, \quad S_2(x) = \sum_{i \sim j} \mathbf{1}_{x_i = x_j} $

### What is $ S_2 $?
- It represents the number of **neighboring pairs** where the spins are the same.
- This is a key statistic because it captures the effect of the interaction parameter $ \beta $, which favors alignment.



## 2. Choosing the Distance Function

Since ABC-Reject compares **simulated** and **observed** data, we define a distance function:

$ d(S(x_{\text{obs}}), S(x_{\text{sim}})) = \left| S(x_{\text{obs}}) - S(x_{\text{sim}}) \right| + \left| S_2(x_{\text{obs}}) - S_2(x_{\text{sim}}) \right| $

### Why this choice?
- $ S(x) $ captures the **overall magnetization** (effect of $ \alpha $).
- $ S_2(x) $ captures **neighboring alignment** (effect of $ \beta $).
- The absolute difference ensures simple, interpretable comparison.
- A **small threshold $ \epsilon $** ensures we accept only close matches.



## 3. ABC-Reject Algorithm Explained

Since computing the likelihood function is intractable, **ABC** estimates parameters using simulation:

1. **Generate observed data** $ x_{\text{obs}} $ from the Ising model with **unknown** $ (\alpha^*, \beta^*) $.
2. **For many iterations**:
   - Sample $ (\alpha, \beta) $ from a **prior distribution**.
   - Simulate $ x_{\text{sim}} $ using the Ising model with $ (\alpha, \beta) $.
   - Compute sufficient statistics $ S_2(x_{\text{sim}}) $.
   - **Accept** $ (\alpha, \beta) $ if the distance is **less than** $ \epsilon $.
3. The set of **accepted** $ (\alpha, \beta) $ values approximates their posterior distribution.



## 4. Why Use ABC-Reject?

- **No need for likelihood computation** (which is intractable due to $ Z(\alpha, \beta) $).
- **Flexible**—can be applied to models with unknown normalizing constants.
- **Drawback**: Inefficient for high-dimensional problems (many rejections).

---


## Why are $ S(x) $ and $ S_2(x) $ sufficient statistics for the Ising Model?

For the Ising model, the parameters we want to estimate are $ \alpha $ and $ \beta $, which affect the magnetization and the interactions between neighboring spins, respectively.

- **Magnetization (summed spins):**

  $$
  S(x) = \sum_i x_i
  $$

  This statistic captures the overall magnetic alignment (effect of $ \alpha $) in the system.

- **Neighbor interactions:**

  $$
  S_2(x) = \sum_{i \sim j} \mathbf{1}_{x_i = x_j}
  $$

  This statistic counts how many neighboring spins are aligned (effect of $ \beta $).


### Intuition:

- The **magnetization statistic** $ S(x) $ captures how many spins are aligned, which is directly influenced by $ \alpha $ (external field).
- The **interaction statistic** $ S_2(x) $ captures how many neighboring spins are aligned, which is directly influenced by $ \beta $ (interaction strength).
- Together, these statistics summarize the system in a way that no additional information (about individual spins or the precise configuration) is needed to estimate $ \alpha $ and $ \beta $.

---

### What Does "Sufficiency" Mean in Practice?

- **In Ising model**: Knowing the total magnetization and the number of aligned neighbors in the system gives us everything we need to estimate the interaction strength $ \beta $ and external field $ \alpha $.
- **For parameter estimation**: Once we have $ S(x) $ and $ S_2(x) $, we can use methods like Maximum Likelihood Estimation (MLE) or Approximate Bayesian Computation (ABC) to find the values of $ \alpha $ and $ \beta $ that best explain the observed data.




RESULTS for $\alpha$ = 0.5 and $\beta$ = 0.8 : 

| $ n $ (System Size) | Number of Samples | $ \epsilon $ (Tolerance) | $ \alpha $ Estimate | $ \beta $ Estimate | Time (s) | MSE |
|---------------------|-------------------|--------------------------|---------------------|--------------------|----------|-----|
| 10                  | 10000             | 0.5                      | 0.43                | 0.96               | 0.00       | 0.12 |
| 20                  | 10000              | 0.5                     | 0.55                | 0.66               | 26       | 0.09 |
| 30                  | 10000              | 0.5                     | 0.92                | 0.71               | 114       | 0.27 |
